In [1]:
import pymatgen
from pymatgen.core.structure import Structure
from pymatgen.ext.matproj import MPRester
import numpy as np

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
mprester = MPRester()

In [3]:
# mp-149 is the Materials Project ID for silicon
si_structure = mprester.query(criteria={'material_id': 'mp-149'}, properties=['structure'])[0]['structure']
si_structure.to(filename='si.cif')

/home/tess/Dropbox/GitHubRepos/pymatgen/pymatgen/io/cif.py:37: UserWarning:

Please install optional dependency pybtex if youwant to extract references from CIF files.



In [4]:
si_structure

Structure Summary
Lattice
    abc : 3.866974622849504 3.866974623775052 3.86697462
 angles : 60.000000032293386 60.00000002437586 60.0000000241681
 volume : 40.888291888494884
      A : 3.34889826 0.0 1.93348731
      B : 1.11629942 3.15737156 1.93348731
      C : 0.0 0.0 3.86697462
PeriodicSite: Si (3.9070, 2.7627, 6.7672) [0.8750, 0.8750, 0.8750]
PeriodicSite: Si (0.5581, 0.3947, 0.9667) [0.1250, 0.1250, 0.1250]

In [8]:
print("There are "+ str(len(si_structure)) + " atoms in the silicon unit cell.")
print(type(si_structure[0])) # this is a pymatgen.core.sites.PeriodicSite
"""
Note there are two coordinate systems that we typically deal with
to describe atomic positions in unit cells
"""
print(si_structure[0].coords) # Cartesian coordinates (the typical x, y, z)
print(si_structure[0].frac_coords) # Fractional coordinates (between 0 and 1)

print(si_structure.lattice)
print(type(si_structure.lattice))

There are 2 atoms in the silicon unit cell.
<class 'pymatgen.core.sites.PeriodicSite'>
[3.90704797 2.76270011 6.76720559]
[0.875 0.875 0.875]
3.348898 0.000000 1.933487
1.116299 3.157372 1.933487
0.000000 0.000000 3.866975
<class 'pymatgen.core.lattice.Lattice'>


In [9]:
atoms = si_structure.get_sites_in_sphere(si_structure[0].coords, 5, include_index=True, include_image=True)
print(atoms[:3])

[(PeriodicSite: Si (0.5581, 2.7627, 4.8337) [-0.1250, 0.8750, 0.8750], 3.8669746228495043, 0, array([-1,  0,  0])), (PeriodicSite: Si (0.5581, 2.7627, 8.7007) [-0.1250, 0.8750, 1.8750], 3.866974622849504, 0, array([-1,  0,  1])), (PeriodicSite: Si (1.6744, 5.9201, 6.7672) [-0.1250, 1.8750, 0.8750], 3.866974624724887, 0, array([-1,  1,  0]))]


In [10]:
def get_coords_of_sites_in_sphere(structure, center_site, radial_cutoff=5., 
                                  center_on_site=True, include_center=False):
    # This function takes care of figuring out which image 
    # (copy across periodic boundaries) of a given atom 
    # is within the sphere.
    atoms_in_sphere = structure.get_sites_in_sphere(center_site.coords,
                                                    radial_cutoff,
                                                    include_index=True, 
                                                    include_image=True)
    
    atom_sites, distances, indices, images = zip(*atoms_in_sphere)
    
    if include_center:
        cart_coords = np.array([atom.coords for atom in atom_sites])
    else:
        cart_coords = np.array([atom.coords for atom in atom_sites if atom != center_site])
    
    if center_on_site:
        cart_coords -= center_site.coords.reshape(-1, 3)
    
    return cart_coords

In [13]:
coords = get_coords_of_sites_in_sphere(si_structure,
                                       si_structure[0], 
                                       radial_cutoff=3, 
                                       center_on_site=True, 
                                       include_center=True)
print(coords)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.23259884e+00  7.89342890e-01 -8.88178420e-16]
 [-4.44089210e-16 -2.36802867e+00 -8.88178420e-16]
 [ 1.11629942e+00  7.89342890e-01 -1.93348731e+00]
 [ 1.11629942e+00  7.89342890e-01  1.93348731e+00]]


In [14]:
# plot coordinates
plot_trace = go.Scatter3d(x=coords[:,0], 
                          y=coords[:,1], 
                          z=coords[:,2],
                          mode="markers")
# data MUST be a list
data = [plot_trace]
layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [17]:
si_dict = si_structure.as_dict()

In [18]:
si_struct_again = Structure.from_dict(si_dict)

In [19]:
si_struct_again

Structure Summary
Lattice
    abc : 3.866974622849504 3.866974623775052 3.86697462
 angles : 60.000000032293386 60.00000002437586 60.0000000241681
 volume : 40.888291888494884
      A : 3.34889826 0.0 1.93348731
      B : 1.11629942 3.15737156 1.93348731
      C : 0.0 0.0 3.86697462
PeriodicSite: Si (3.9070, 2.7627, 6.7672) [0.8750, 0.8750, 0.8750]
PeriodicSite: Si (0.5581, 0.3947, 0.9667) [0.1250, 0.1250, 0.1250]